<a href="https://colab.research.google.com/github/Benktree/Benktree/blob/main/2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files

# 运行此代码后，手动选择 Excel 文件上传
uploaded = files.upload()

# 获取文件名
file_path = list(uploaded.keys())[0]
print(f"✅ 文件 {file_path} 上传成功！")

import pandas as pd

# 读取上传的 Excel 文件
df = pd.read_excel(file_path)

# 确保数据包含 "City", "Latitude", "Longitude" 列
required_columns = {"City", "Latitude", "Longitude"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"❌ Excel 文件缺少必要的列: {required_columns - set(df.columns)}")

print("✅ Excel 数据加载成功！")

import numpy as np
import openrouteservice

# ✅ 设置 OpenRouteService API Key
API_KEY = "5b3ce3597851110001cf6248afe4f39f8c7d41e8a0a42d5fcc742d3c"  # ⚠️ 请替换为你的 API Key
client = openrouteservice.Client(key=API_KEY)

# ✅ 提取城市和坐标
cities = df[["City", "Latitude", "Longitude"]].dropna()
locations = list(zip(cities["Longitude"], cities["Latitude"]))  # OpenRouteService 使用 (lon, lat) 结构

# ✅ 调用 API 计算距离矩阵
try:
    response = client.distance_matrix(
        locations=locations,
        profile="driving-car",  # 🚗 确保 API 计算的是公路行驶距离
        metrics=["distance"],   # 计算距离
        units="m"               # API 默认返回米，我们稍后转换为 km
    )

    # ✅ 确保 API 返回的是 "driving-car"
    metadata = response.get("metadata", {})
    print(f"✅ OpenRouteService API Metadata: {metadata}")

    # ✅ 解析结果并转换为公里
    distances_matrix = np.array(response["distances"]) / 1000  # 转换为 km
    distance_df = pd.DataFrame(distances_matrix, index=cities["City"], columns=cities["City"])

    # ✅ 选取部分数据进行验证
    city1, city2 = 0, 1  # 选取前两个城市
    print(f"🛣️ 从 {cities.iloc[city1]['City']} 到 {cities.iloc[city2]['City']} 的公路距离：{distances_matrix[city1, city2]:.2f} km")

    # ✅ 保存 Excel 文件
    output_path = "Distance_Matrix.xlsx"
    distance_df.to_excel(output_path)
    print(f"✅ 距离矩阵已生成: {output_path}")

    # ✅ 提供文件下载
    files.download(output_path)

except Exception as e:
    print(f"❌ 发生错误: {e}")




Saving European_Cities.xlsx to European_Cities (3).xlsx
✅ 文件 European_Cities (3).xlsx 上传成功！
✅ Excel 数据加载成功！
✅ OpenRouteService API Metadata: {'attribution': 'openrouteservice.org | OpenStreetMap contributors', 'service': 'matrix', 'timestamp': 1738527173678, 'query': {'locations': [[2.3522, 48.8566], [4.8357, 45.764], [5.3698, 43.2965], [13.405, 52.52], [11.582, 48.1351], [8.6821, 50.1109], [16.3738, 48.2082], [4.3517, 50.8503], [4.4025, 51.2194], [-6.2603, 53.3498], [12.4964, 41.9028], [9.19, 45.4642], [14.2681, 40.8518], [6.1319, 49.6116], [4.9041, 52.3676], [4.4777, 51.9244], [-9.1393, 38.7223], [-8.6291, 41.1579], [-3.7038, 40.4168], [2.1734, 41.3851], [8.5417, 47.3769], [6.1432, 46.2044], [-0.1278, 51.5074], [-2.2426, 53.4808], [-1.8904, 52.4862], [12.5683, 55.6761], [24.7536, 59.437], [24.9384, 60.1699], [24.1052, 56.9496], [25.2797, 54.6872], [10.7522, 59.9139], [21.0122, 52.2297], [19.945, 50.0647], [18.0686, 59.3293], [11.9746, 57.7089], [23.3219, 42.6977], [15.9819, 45.815], 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>